In [ ]:
import re
import pandas as pd
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import numpy as np


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from nltk.corpus import stopwords

**Part A**


In [ ]:
df = pd.read_json("https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json")

In [ ]:
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [ ]:
df.dtypes

content         object
target           int64
target_names    object
dtype: object

In [ ]:
#sum of entries
df.shape[0]

11314

In [ ]:
#target_name is different topics of the posts
#their distribution
df.target_names.value_counts(normalize=True)


rec.sport.hockey            0.053032
soc.religion.christian      0.052943
rec.motorcycles             0.052855
rec.sport.baseball          0.052766
sci.crypt                   0.052590
rec.autos                   0.052501
sci.med                     0.052501
comp.windows.x              0.052413
sci.space                   0.052413
comp.os.ms-windows.misc     0.052236
sci.electronics             0.052236
comp.sys.ibm.pc.hardware    0.052148
misc.forsale                0.051706
comp.graphics               0.051617
comp.sys.mac.hardware       0.051087
talk.politics.mideast       0.049850
talk.politics.guns          0.048259
alt.atheism                 0.042425
talk.politics.misc          0.041100
talk.religion.misc          0.033322
Name: target_names, dtype: float64

In [ ]:
#first content value and target name
print(df.content[0],df.target_names[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----




 rec.autos


*Which business question can this dataset address?*


This dataset can be used to tag the observed data (post/thread) into the correct target group (topic).

**Part B**

In [ ]:
re.sub("(?m)^(From:|Article-I.D.:|Organization:|Lines:|Nntp-Posting-Host:|Distribution:|Reply-To:|X-Newsreader:|Expires:|\s*-+).*\n","",df.content[0])

'Subject: WHAT car is this!?\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n\n\n\n\n'

In [ ]:
#remove the lines...
regex = "(?m)^(From:|Article-I.D.:|Organization:|Lines:|Nntp-Posting-Host:|Distribution:|Reply-To:|X-Newsreader:|Expires:|\s*-+).*\n"
regex_2 = "Subject: |Summary: |Keywords:"
data=[]
for i in range(len(df.content)-1):
  i=i+1
  data.append(re.sub(regex,"",re.sub(regex_2,"",df.content[i],flags=re.IGNORECASE)))
  

**Part C**

In [ ]:
data = [strip_multiple_whitespaces(strip_punctuation(strip_numeric(str(text)))) for text in data]

`strip_numeric`: Remove digits from text

`strip_punctuation`: Replace punctuation characters with spaces

`strip_multiple_whitespaces`: Remove repeating whitespace characters (spaces, tabs, line breaks) from text and turns tabs & line breaks into spaces usin



In [ ]:
#Transform all letters to lower case ones and override data with the result
data = [str(text).lower() for text in data]

**Part D**

In [ ]:
#stop gensim
stop_gensim = list(STOPWORDS)
stop_gensim.sort()
print(stop_gensim)
print(len(STOPWORDS))

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'did', 'didn', 'do', 'does', 'doesn', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found

In [ ]:
#stop gensim
stop_nltk = stopwords.words("english")
stop_nltk.sort()
print(stop_nltk)
print(len(stop_nltk))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

We observe that nltk has fewer stopwords compared to gensim.

In [ ]:
#stop words which are in gensim but not nltk
print(set(stop_gensim)-set(stop_nltk))
print(len(set(stop_gensim)-set(stop_nltk)))

{'seeming', 'anything', 'others', 'wherever', 'de', 'thick', 'two', 'using', 'serious', 'beyond', 'whereafter', 'throughout', 'front', 'already', 'regarding', 'became', 'none', 'becomes', 'namely', 'less', 'therein', 'whereby', 'though', 'mostly', 'say', 'ie', 'somehow', 'thin', 'nothing', 'bottom', 'noone', 'someone', 'except', 'many', 'whereupon', 'enough', 'ltd', 'cry', 'nine', 'wherein', 'somewhere', 'sincere', 'besides', 'con', 'fill', 'top', 'toward', 'nevertheless', 'yet', 'find', 'onto', 'go', 'latterly', 'whether', 'hereupon', 'several', 'five', 'perhaps', 'elsewhere', 'various', 'describe', 'fify', 'please', 'take', 'whence', 'hereby', 'anywhere', 'twelve', 'otherwise', 'detail', 'behind', 'never', 'made', 'eleven', 'system', 'almost', 'side', 'mill', 'thereupon', 'along', 'four', 'rather', 'anyhow', 'towards', 'back', 'whenever', 'fifteen', 'beside', 'amount', 'per', 'move', 'across', 'really', 'due', 'might', 'sixty', 'would', 'six', 'fire', 'hereafter', 'first', 'either', 

In [ ]:
#stop words which are in nltk but not gensim
print(set(stop_nltk)-set(stop_gensim))
print(len(set(stop_nltk)-set(stop_gensim)))

{'wouldn', 'needn', "you're", 'hasn', 'o', 'd', "that'll", "wasn't", "you'd", "hasn't", 'aren', "wouldn't", 'won', 'll', 'couldn', "isn't", 'theirs', "don't", 'shan', "mustn't", 't', "weren't", "doesn't", 'hadn', 'having', 'mustn', "she's", "won't", "mightn't", 's', 'shouldn', 'm', "it's", 'wasn', 've', "shouldn't", "should've", "haven't", "you've", 'y', "hadn't", "you'll", 'mightn', "shan't", 'haven', "didn't", 'ain', 'weren', "needn't", 'ma', "aren't", 'isn', "couldn't"}
53


Here, also observe that stop words in nltk also contain punctuation

In [ ]:
#Remove the stopwords using gensim’s remove_stopwords and override data
data = [remove_stopwords(str(text)) for text in data]

In [ ]:
data_shorted = [strip_short(str(text)) for text in data]

`Strip_short`: Remove words with length lesser than minsize from default 3.

**Part E**

In [ ]:
data_stemmed = stem_text(str(data))

`stem_text`: Transform text into lowercase and stem it its stem or root format

In [ ]:
#lemmatization with SpaCy
nlp = en_core_web_sm.load()

In [ ]:
data_lemma = []
for text in data:
    doc = nlp(str(text))
    data_lemma.append(" ".join([token.lemma_ for token in doc]))

In [ ]:
data_lemma[1]

'pb questions folk mac plus finally give ghost weekend start life k way sooo m market new machine bit sooner intend m looking pick powerbook maybe bunch question hopefully somebody answer anybody know dirt round powerbook introduction expect d hear c suppose appearence summer haven t hear anymore t access macleak wonder anybody info anybody hear rumor price drop powerbook line like one duo s go recently s impression display probably swing get mb disk t feel well display yea look great store wow good solicit opinion people use day day worth take disk size money hit active display realize real subjective question ve play machine store breifly figure opinion somebody actually use machine daily prove helpful hellcat perform thank bunch advance info email will post summary news reading time premium final corner tom willis twillis ecn purdue edu purdue electrical engineering conviction dangerous enemy truth lie f w nietzsche'

In [ ]:
with open('/content/drive/MyDrive/Colab/data_stemmed.pkl', "wb") as output_file:
  pickle.dump(data_stemmed, output_file)

In [ ]:
with open('/content/drive/MyDrive/Colab/data_lemma.pkl', "wb") as output_file:
  pickle.dump(data_lemma, output_file)

**Extra Challenge**

Compute the score separately by chapter of https://www.tidytextmining.com/

In [ ]:
!pip install gutenberg
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import pandas as pd
import urllib
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import math

In [ ]:
TWSM_path = "/content/drive/MyDrive/Colab/" 
bing = pd.read_csv(TWSM_path)


In [ ]:
#text transform for sentiment score
def tf(t):
  (unique, wordcounts)=np.unique(t.split(),return_counts=True)
  data = pd.DataFrame(unique,wordcounts).reset_index().rename(columns={"index":"wordcounts",0:"words"}).sort_values(by="wordcounts",ascending=False)
  data["rank"]=data["wordcounts"].rank(ascending=True)

  return data
  print(data)


In [ ]:
#get all urls for chapters in tidytextmining
url = 'https://www.tidytextmining.com/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
values=[]
link_tidytext=[]
for link in soup.find_all('a'):
    a=(link.get('href'))
    b=re.search(r'html$',a)
    if b:
      values.append(a)
for v in values:
  c = url+v
  link_tidytext.append(c)


In [ ]:
print("Sentiment analysis for each of the chapter of Tidy Text Mining:")
for url in link_tidytext:
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  textnew = tf(text)
  dic  = pd.read_csv(TWSM_path)
  #sentiment_merge = pd.merge(dic, df_words, how="inner", on="words")
  sentiment_merge = pd.merge(dic, textnew, left_on="word", right_on="words")
  sentiment_merge["total_value"] = sentiment_merge["value"] * sentiment_merge["wordcounts"]
  ovr_score = np.mean(sentiment_merge["total_value"])
  ovr_score = float(ovr_score)
  if not math.isnan(ovr_score):
    print("Chapter " + url + ": ",ovr_score)

Sentiment analysis for each of the chapter of Tidy Text Mining:
Chapter https://www.tidytextmining.com/preface.html:  2.5925925925925926
Chapter https://www.tidytextmining.com/tidytext.html:  1.21875
Chapter https://www.tidytextmining.com/sentiment.html:  0.6666666666666666
Chapter https://www.tidytextmining.com/tfidf.html:  2.5833333333333335
Chapter https://www.tidytextmining.com/ngrams.html:  0.3333333333333333
Chapter https://www.tidytextmining.com/dtm.html:  0.75
Chapter https://www.tidytextmining.com/topicmodeling.html:  0.5476190476190477
Chapter https://www.tidytextmining.com/twitter.html:  1.76
Chapter https://www.tidytextmining.com/nasa.html:  3.2580645161290325
Chapter https://www.tidytextmining.com/usenet.html:  0.8085106382978723
Chapter https://www.tidytextmining.com/preface.html:  2.5925925925925926
